In [1]:
import pandas

In [2]:
fpath = 'C:\\Users\\watson\\Documents\\GitHub\\SabberStone-master\\Sabber_Work_2019F\\thesis-output\\StrategySearchResults\\Warlock_Net_AA_sm\\logs\\fittest_log.csv'
f = open(fpath)
lines = f.readlines()
f.close()

In [3]:
for line in lines:
    print(line)

Individual,Emitter,Generation,WinCount,AverageHealthDifference,DamageDone,NumTurns,CardsDrawn,HandSize,ManaSpent,ManaWasted,StrategyAlignment,Dust,DeckManaSum,DeckManaVariance,NumMinionCards,NumSpellCards,S0:WinCount,S0:Alignment,Weight:0,Weight:1,Weight:2,Weight:3,Weight:4,Weight:5,Weight:6,Weight:7,Weight:8,Weight:9,Weight:10,Weight:11,Weight:12,Weight:13,Weight:14,Weight:15,Weight:16,Weight:17,Weight:18,Weight:19,Weight:20,Weight:21,Weight:22,Weight:23,Weight:24,Weight:25,Weight:26,Weight:27,Weight:28,Weight:29,Weight:30,Weight:31,Weight:32,Weight:33,Weight:34,Weight:35,Weight:36,Weight:37,Weight:38,Weight:39,Weight:40,Weight:41,Weight:42,Weight:43,Weight:44,Weight:45,Weight:46,Weight:47,Weight:48,Weight:49,Weight:50,Weight:51,Weight:52,Weight:53,Weight:54,Weight:55,Weight:56,Weight:57,Weight:58,Weight:59,Weight:60,Weight:61,Weight:62,Weight:63,Weight:64,Weight:65,Weight:66,Weight:67,Weight:68,Weight:69,Weight:70,Weight:71,Weight:72,Weight:73,Weight:74,Weight:75,Weight:76,Weight:77,

In [65]:
firstline = lines[0].strip('\n').split(',')
firstWeightIndex = 0
lastWeightIndex = -1
c = -1 #this will turn out to 18 -> there are 18+1 index positions before weights come in
for i in range(len(firstline)):
    if 'Weight' not in firstline[i]:
        c += 1
    elif 'Weight:0' in firstline[i]:
        firstWeightIndex = i
    lastWeightIndex += 1
    #else:
    #    print(i, firstline[i])
numWeights = lastWeightIndex - c
print('firstWeightIndex', firstWeightIndex, firstline[firstWeightIndex])
print('lastWeightIndex', lastWeightIndex)
print('c', c)
print('numWeights', numWeights)

firstWeightIndex 19 Weight:0
lastWeightIndex 127
c 18
numWeights 109


In [62]:
c

19

In [66]:
myWeights = []
for i in range(1, len(lines)):
    line = lines[i].strip('\n')
    lineSplit = line.split(',')
    lineWeights = [0 for i in range(numWeights)]
    #print(firstWeightIndex, len(lineSplit))
    for j in range(firstWeightIndex, len(lineSplit)):
        try:
            lineWeights[j-c-1] = float(lineSplit[j])
        except IndexError as e:
            print('Error at index {}'.format(j))
    myWeights.append(lineWeights)
    

In [67]:
len(myWeights[0])

109

In [57]:
j-c-1

108

In [58]:
j-19

108

In [5]:
test = '''
                string[] lines = System.IO.File.ReadAllLines(weights_file);

				int firstWeightIndex = 0;
				int lastWeightIndex = 0;

				foreach (string line in lines)
				{ //iterate over each line of fittest_log.csv
					string[] myRowLine = line.Split(","); //this is the line of the csv split
					var weights = new double[lastWeightIndex - firstWeightIndex + 1];

					for (int i = 0; i < myRowLine.Count(); i++)
					{ //iterate over the index values for the line
						if (myRowLine[i].Contains("Weight:0")) // if (in the first line) there is Weight:0
						{
							//assign firstWeightIndex and lastWeightIndex for every line of the csv
							firstWeightIndex = i;
							lastWeightIndex = myRowLine.Count() - 1;
							Console.WriteLine("Index Weight 0: " + firstWeightIndex);
							Console.WriteLine(myRowLine[i]);

						}
						else if (firstWeightIndex != 0 && i >= firstWeightIndex)
						{ // otherwise, if the firstWeightIndex != 0 and the current index i is >= firstWeightIndex
							if (!myRowLine[i].Contains("Weight")) //AND we are on a line after line 0
							{ // we have a weight, assign it into the weights vector
								weights[i - 19] = Double.Parse(myRowLine[i]);
							}

							Console.WriteLine("Weight:" + i + ", " + myRowLine[i]);
						}
					}
					// add the weights from a line in fittest_log.csv to the myWeights vector
					myWeights.Add(weights);
				}

'''

In [1]:
import numpy as np
import scipy.stats as scs
import math
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib
import matplotlib.pyplot as plt

import re
import argparse
import logging

import os
# import glob
import sys
from pathlib import Path

from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

import gamework
import game_filter

In [3]:
import numpy as np
import pandas as pd
import matplotlib

import argparse
import logging

import os
import sys
from pathlib import Path
from gamework import GameFilter

In [5]:
machine = 'laptop'
sub_dir = 'C:\\Users\\watson\\Documents\\SabberMatchups\\AvsNNA_1k02'
output_folder = 'C:\\Users\\watson\\Documents\\GitHub\\SabberStone-master\\Sabber_Work_2019F\\thesis-output\\SabberGameMatchups'

if machine == 'laptop':
    # laptop directory
    directory = 'C:\\Users\\watson\\Documents\\GitHub\\SabberStone-master\\Sabber_Work_2019F\\thesis-output\\' + output_folder + '\\'
elif machine == 'desktop':
    # desktop directory
    # directory = 'C:\\Users\\watson\\Documents\\SabberStone 2019\\Sabber_Work_2019F\\thesis-output\\'
    directory = 'C:\\Users\\watson\\Documents\\SabberStone 2019\\Sabber_Work_2019F\\thesis-output\\' + output_folder + '\\'
elif machine == 'mixr':
    directory = 'C:\\Users\\Main\\Documents\\GitHub\\Sabber_Work_2019F\\thesis-output\\' + output_folder + '\\'
else:
    logging.warning('UNEXPECTED OPTION IN CMD, CONFIG PROPERLY')
    sys.exit(0)

try:
    os.mkdir(directory)
except OSError:
    print("Creation of the directory %s failed" % directory)
else:
    print("Successfully created the directory %s " % directory)

new_sub_dir = sub_dir.split('\\')[-1] + '_Compiled'
if sub_dir[:2] == 'C:':
    full_directory = sub_dir
else:
    full_directory = '{}{}\\'.format(directory, sub_dir)
full_output_directory = '{}{}\\'.format(directory, new_sub_dir)
#print(full_directory)
#print(full_output_directory)
#sys.exit(0)
try:
    os.mkdir(full_output_directory)
except OSError:
    print("Creation of the directory %s failed" % full_output_directory)
else:
    print("Successfully created the directory %s " % full_output_directory)
list_subfolders_with_paths = [f.path for f in os.scandir(full_directory) if f.is_dir()]
for f1 in list_subfolders_with_paths:
    logging.debug(f1)
    list_matchup_folders = [f.path for f in os.scandir(f1) if f.is_dir()]
    for f2 in list_matchup_folders:
        logging.info('Current folder: {}'.format(f2))
        game_counter = 0
        matchup_data = {}
        logging.debug(f2)
        csv_file_name = '-'.join(f2.split('\\')[-2:]) + '.csv'
        csv_output_path = full_output_directory + csv_file_name
        logging.debug(csv_output_path)
        pathlist = Path(f2).glob('**\\*.txt')
        for game_name in pathlist:
            logging.debug(game_name)
            # game_csv_name = game_name.replace('.txt', '.csv')
            # game_plot_name = game_name.replace('.txt', '.png')
            # logging.info(game_name)
            # logging.info(game_csv_name)
            # logging.info(game_plot_name)

            game_obj = GameFilter(game_name, game_counter)
            game_obj.parse_file()
            # logging.info(len(game_obj.games_data))
            matchup_data.update(game_obj.games_data)
            game_counter += game_obj.num_games
        # logging.info('Number of turns: {}'.format(len(game_obj.end_of_turn_data)))
        # game_obj.plot_data(game_plot_name)
        
        ending_df = pd.concat(list(matchup_data.values()))
        ending_df.reset_index(inplace=True, drop=True)
        with open(csv_output_path, 'w') as f:
            ending_df.to_csv(csv_output_path)
        logging.info('Check {}'.format(csv_output_path))

        logging.error('THERE WAS A PROBLEM CONCATTING DFs')
        logging.error('NUMBER OF PAIRS {}'.format(len(matchup_data)))


Creation of the directory C:\Users\watson\Documents\GitHub\SabberStone-master\Sabber_Work_2019F\thesis-output\C:\Users\watson\Documents\GitHub\SabberStone-master\Sabber_Work_2019F\thesis-output\SabberGameMatchups\ failed
Creation of the directory C:\Users\watson\Documents\GitHub\SabberStone-master\Sabber_Work_2019F\thesis-output\C:\Users\watson\Documents\GitHub\SabberStone-master\Sabber_Work_2019F\thesis-output\SabberGameMatchups\AvsNNA_1k02_Compiled\ failed


OSError: [Errno 22] Invalid argument: 'C:\\Users\\watson\\Documents\\GitHub\\SabberStone-master\\Sabber_Work_2019F\\thesis-output\\C:\\Users\\watson\\Documents\\GitHub\\SabberStone-master\\Sabber_Work_2019F\\thesis-output\\SabberGameMatchups\\AvsNNA_1k02_Compiled\\Pizza-NNPizza.csv'

In [6]:
ending_df

,TURN_NO,P1_HEALTH,P2_HEALTH,CURRENT_PLAYER,AMOUNTHEALEDTHISTURN,NUMATTACKSTHISTURN,NUMCARDSDRAWNTHISTURN,NUMCARDSPLAYEDTHISTURN,NUMCARDSTODRAW,NUMELEMENTALSPLAYEDLASTTURN,NUMELEMENTALSPLAYEDTHISTURN,NUMFRIENDLYMINIONSTHATATTACKEDTHISTURN,NUMFRIENDLYMINIONSTHATDIEDTHISTURN,NUMMINIONSPLAYEDTHISTURN,NUMMINIONSPLAYERKILLEDTHISTURN,NUMOPTIONSPLAYEDTHISTURN,NUMSECRETSPLAYEDTHISGAME,NUMSPELLSPLAYEDTHISGAME,NUMTIMESHEROPOWERUSEDTHISGAME,REMAININGMANA,TOTALMANASPENTTHISGAME,USEDMANATHISTURN,GAME_COUNTER
0,1,30,30,P1 Pizza,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,1,1,1
1,2,30,30,P2 NNPizza,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,1,1,1
2,3,30,30,P1 Pizza,0,0,1,2,0,0,0,1,0,2,0,3,0,0,0,0,3,2,1
3,4,28,30,P2 NNPizza,0,0,1,3,0,0,0,1,0,2,0,4,0,1,0,0,4,2,1
4,5,28,30,P1 Pizza,0,0,4,2,0,0,0,3,2,1,4,5,0,1,0,0,6,3,1
5,6,25,30,P2 NNPizza,0,0,1,2,0,0,0,1,1,1,1,3,0,2,0,0,7,3,1
6,7,25,22,P1 Pizza,0,0,1,3,0,0,0,2,1,2,2,5,0,2,0,0,10,4,1
7,8,16,17,P2 NNPizza,0,0,2,2,0,0,0,3,0,2,0,5,0,2,1,0,11,4,1
8,9,16,6,P1 Pizza,0,0,1,1,0,0,0,3,0,1,0,4,0,2,0,4,11,1,1
9,10,12,6,P2 NNPizza,0,0,1,1,0,0,0,5,2,0,6,6,0,3,1,1,15,4,1


In [7]:
csv_output_path

'C:\\Users\\watson\\Documents\\GitHub\\SabberStone-master\\Sabber_Work_2019F\\thesis-output\\C:\\Users\\watson\\Documents\\GitHub\\SabberStone-master\\Sabber_Work_2019F\\thesis-output\\SabberGameMatchups\\AvsNNA_1k02_Compiled\\Pizza-NNPizza.csv'

In [8]:
#python game_filter.py laptop SabberGameMatchups C:\Users\watson\Documents\SabberMatchups\AvsNNA_1k02